In [ ]:
! pip install requests beautifulsoup4

In [ ]:
import requests
import csv
import time

API_URL = 'https://publicapi.traffy.in.th/teamchadchart-stat-api/geojson/v1'
LIMIT = 1000
TOTAL = 20000
params_template = {
    'output_format': 'json',
    'name': 'รัชปัศย์ แบนประเสริฐ',
    'org': 'CU',
    'purpose': 'ทำ project วิชา 2110446 DSDE 2024/2 ',
    'email': 'traffyteam@gmail.com',
    'limit': LIMIT,
    'offset': 0,
    'sort': 'DESC'
}

seen_ticket_ids = set()
seen_comments = set()

with open('fondue_scraping.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['ticket_id', 'type', 'organization', 'comment',
                  'coords', 'address', 'subdistrict', 'district', 'province',
                  'timestamp', 'state', 'star', 'count_reopen', 'last_activity']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for offset in range(0, TOTAL, LIMIT):
        params = params_template.copy()
        params['offset'] = offset

        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])
            print(f'Offset {offset}: Got {len(features)} records')
            for item in features:
                props = item['properties']
                coords = item['geometry']['coordinates']
                ticket_id = props.get('ticket_id')
                comment = props.get('description', '').strip()

                if ticket_id in seen_ticket_ids or comment in seen_comments:
                    continue

                seen_ticket_ids.add(ticket_id)
                if comment:
                    seen_comments.add(comment)

                # ดึง type และทำ format เป็น {type1,type2}
                type_list = props.get('problem_type_fondue', [])
                type_field = f"{{{', '.join(type_list)}}}" if type_list else ''

                # ดึง org และลบ 'กรุงเทพมหานคร' ออก
                org_list = [org for org in props.get('org', []) if org != 'กรุงเทพมหานคร']
                org_field = ', '.join(org_list)

                writer.writerow({
                    'ticket_id': ticket_id,
                    'type': type_field,
                    'organization': org_field,
                    'comment': comment,
                    'coords': f"{coords[0]}, {coords[1]}" if coords else '',
                    'address': props.get('address', ''),
                    'subdistrict': props.get('subdistrict', ''),
                    'district': props.get('district', ''),
                    'province': props.get('province', ''),
                    'timestamp': props.get('timestamp', ''),
                    'state': props.get('state', ''),
                    'star': props.get('star'),
                    'count_reopen': props.get('count_reopen'),
                    'last_activity': props.get('last_activity', '')
                })
        else:
            print(f'Failed to fetch offset {offset}: {response.status_code}')

        time.sleep(1)


Offset 0: Got 1000 records
Offset 1000: Got 1000 records
Offset 2000: Got 1000 records
Offset 3000: Got 1000 records
Offset 4000: Got 1000 records
Offset 5000: Got 1000 records
Offset 6000: Got 1000 records
Offset 7000: Got 1000 records
Offset 8000: Got 1000 records
Offset 9000: Got 1000 records
Offset 10000: Got 1000 records
Offset 11000: Got 1000 records
Offset 12000: Got 1000 records
Offset 13000: Got 1000 records
Offset 14000: Got 1000 records
Offset 15000: Got 1000 records
Offset 16000: Got 1000 records
Offset 17000: Got 1000 records
Offset 18000: Got 1000 records
Offset 19000: Got 1000 records
